In [23]:
%matplotlib inline

import cv2
import csv
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import sys
sys.path.append('/home/steve/git/CreateNormals/')
from python.calc_normals import NormalCalculation

In [24]:
label_nyu_mapping = dict()
label_nyu_mapping[0] = 0
with open('/media/steve/Lilim/scannetv2-labels.combined.tsv') as tsvfile:
    reader = csv.reader(tsvfile, delimiter='\t')
    start = True
    for row in reader:
        print(row[0] + ', ' + row[1] + ', ' + row[4] + ', ' + row[3])
        #print(row)
        if not start:
            label_nyu_mapping[int(row[0])] = int(row[4])
        start = False

id, raw_category, nyu40id, count
1, wall, 1, 8277
2, chair, 5, 4646
22, books, 23, 1678
3, floor, 2, 1553
5, door, 8, 1483
1163, object, 40, 1313
16, window, 9, 1209
4, table, 7, 1170
56, trash can, 39, 1090
13, pillow, 18, 937
15, picture, 11, 862
41, ceiling, 22, 806
26, box, 29, 775
161, doorframe, 8, 768
19, monitor, 40, 765
7, cabinet, 3, 731
9, desk, 14, 680
8, shelf, 15, 641
10, office chair, 5, 595
31, towel, 27, 570
6, couch, 6, 502
14, sink, 34, 488
48, backpack, 40, 479
28, lamp, 35, 419
11, bed, 4, 370
18, bookshelf, 10, 360
71, mirror, 19, 349
21, curtain, 16, 347
40, plant, 40, 331
52, whiteboard, 30, 327
96, radiator, 39, 322
22, book, 23, 318
29, kitchen cabinet, 3, 310
49, toilet paper, 40, 291
29, kitchen cabinets, 3, 289
23, armchair, 5, 281
63, shoes, 40, 272
24, coffee table, 7, 258
17, toilet, 33, 256
47, bag, 37, 252
32, clothes, 21, 248
46, keyboard, 40, 246
65, bottle, 40, 226
97, recycling bin, 39, 225
34, nightstand, 32, 224
38, stool, 40, 221
33, tv, 25, 219

In [25]:
def _read_images_function(image_file, depth_file, label_file):
    image_decoded = cv2.imread(image_file.decode())
    depth_decoded = cv2.imread(depth_file.decode(), cv2.IMREAD_ANYDEPTH)
    label_decoded = cv2.imread(label_file.decode(), cv2.IMREAD_ANYDEPTH)
    label_nyu = np.array([label_nyu_mapping[x] for x in label_decoded.flatten()])
    label_nyu = label_nyu.reshape((480,640))
    #label_decoded = cv2.resize(cv2.imread(label, cv2.IMREAD_ANYDEPTH), (640,480), interpolation=cv2.INTER_NEAREST)
    return image_decoded, depth_decoded, label_decoded

# Use standard TensorFlow operations to resize the image to a fixed shape.
def _resize_function(image_decoded, depth_decoded, label_decoded):
    image_decoded.set_shape([None, None, None])
    depth_decoded.set_shape([None, None, None])
    label_decoded.set_shape([None, None, None])
    image_resized = tf.image.resize(image_decoded, [640, 480])
    depth_resized = tf.image.resize(depth_decoded, [640, 480], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    label_resized = tf.image.resize(label_decoded, [640, 480], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    return image_resized, depth_resized, label_resized

In [26]:
image_files = []
depth_files = []
label_files = []
with open('dataset_test.txt', 'r') as text_file:
    for line in text_file:
        splits = line.split(',')
        image_files.append(splits[0])
        depth_files.append(splits[1])
        label_files.append(splits[2])

dataset = tf.data.Dataset.from_tensor_slices((image_files, depth_files, label_files))
dataset = dataset.map(
    lambda image_file, depth_file, label_file: tuple(tf.compat.v1.py_func(
        _read_images_function, [image_file, depth_file, label_file], [tf.uint8, tf.uint16, tf.uint16])))
dataset = dataset.map(_resize_function)

In [27]:
print dataset

<MapDataset shapes: ((640, 480, None), (640, 480, None), (640, 480, None)), types: (tf.float32, tf.uint16, tf.uint16)>


In [28]:
tmp = cv2.imread('/media/steve/Lilim/scans/scene0642_01/label-filt/3893.png', cv2.IMREAD_ANYDEPTH)
print(tmp)

[[ 1  1  1 ...  0  0  0]
 [ 1  1  1 ...  0  0  0]
 [ 1  1  1 ...  0  0  0]
 ...
 [11 11 11 ...  3  3  3]
 [11 11 11 ...  3  3  3]
 [11 11 11 ...  3  3  3]]


In [29]:
print tf.__version__

2.0.0-alpha0
